# Sector Momentum Analysis

In [1]:
import pandas as pd

## Extract sector table data from etfdb.com

In [2]:
url = 'https://etfdb.com/etfs/sector/'
df = pd.read_html(url)[0].drop([11], axis=0)

In [3]:
sectors = list(df['Sector'])
sectors = [x.lower() for x in sectors]
sectors = [x.replace(' ','-') for x in sectors]
sectors = [x.replace('discretionary', 'discretionaries') for x in sectors]
sectors


['energy',
 'consumer-discretionaries',
 'financials',
 'healthcare',
 'industrials',
 'technology',
 'materials',
 'consumer-staples',
 'telecom',
 'utilities',
 'real-estate']

## Retrieve Top ETFs from Each Sector

In [10]:
top_etfs = {}
for s in sectors:
    sector_url = url + s + '/'
    sector_data = pd.read_html(sector_url)[0]
    etf_symbol = sector_data.loc[0,'Symbol']
    etf_name = sector_data.loc[0,'ETF Name']
    etf_industry = sector_data.loc[0, 'Industry']
    etf_aum = sector_data.loc[0, 'Total Assets ($MM)']
    top_etfs[s] = {}
    top_etfs[s]['symbol'] = etf_symbol
    top_etfs[s]['name'] = etf_name
    top_etfs[s]['industry'] = etf_industry
    top_etfs[s]['aum'] = etf_aum
    
top_etfs

{'energy': {'symbol': 'XLE',
  'name': 'Energy Select Sector SPDR Fund',
  'industry': 'Oil & Gas Exploration & Production',
  'aum': '$22,101.90'},
 'consumer-discretionaries': {'symbol': 'XLY',
  'name': 'Consumer Discretionary Select Sector SPDR Fund',
  'industry': 'Broad Consumer Discretionary',
  'aum': '$19,220.10'},
 'financials': {'symbol': 'XLF',
  'name': 'Financial Select Sector SPDR Fund',
  'industry': 'Broad Financials',
  'aum': '$36,592.80'},
 'healthcare': {'symbol': 'XLV',
  'name': 'Health Care Select Sector SPDR Fund',
  'industry': 'Broad Healthcare',
  'aum': '$24,833.30'},
 'industrials': {'symbol': 'XLI',
  'name': 'Industrial Select Sector SPDR Fund',
  'industry': 'Broad Industrials',
  'aum': '$19,135.00'},
 'technology': {'symbol': 'QQQ',
  'name': 'Invesco QQQ',
  'industry': 'Broad Technology',
  'aum': '$155,470.00'},
 'materials': {'symbol': 'GDX',
  'name': 'VanEck Vectors Gold Miners ETF',
  'industry': 'Gold Miners',
  'aum': '$14,080.20'},
 'consume

In [17]:
convert_df = {'Symbol': [], 'Sector':[], 'Industry': []}
for etf in top_etfs.keys():
    convert_df['Symbol'].append(top_etfs[etf]['symbol'])
    convert_df['Sector'].append(etf)
    convert_df['Industry'].append(top_etfs[etf]['industry'])
    #convert_df['Momentum'].append(top_etfs[etf]['symbol'])

df = pd.DataFrame(convert_df)
df

,Symbol,Sector,Industry
0,XLE,energy,Oil & Gas Exploration & Production
1,XLY,consumer-discretionaries,Broad Consumer Discretionary
2,XLF,financials,Broad Financials
3,XLV,healthcare,Broad Healthcare
4,XLI,industrials,Broad Industrials
5,QQQ,technology,Broad Technology
6,GDX,materials,Gold Miners
7,XLP,consumer-staples,Broad Consumer Staples
8,VOX,telecom,Broad Telecom
9,XLU,utilities,Broad Utilities
